<a href="https://colab.research.google.com/github/kellyshreeve/QA-Chatbot/blob/main/Copy_of_QA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Question Answer Models**

[Llama 2](#llama)

## **Import Packages and Data**

In [ ]:
! pip install transformers
! pip install sentence_transformers
! pip install accelerate
! pip install langchain
! pip install ctransformers
! pip install faiss-gpu
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b9eedc28746e760a8af35188912a2e1785644d071fa1d92aadde30cad5bcbd0a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/

In [6]:
import time
from tqdm.auto import tqdm
import pickle
import accelerate
import json
import textwrap

import math
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords

import torch
import transformers
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from sentence_transformers import SentenceTransformer

from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from langchain.memory import ConversationBufferMemory

from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

import modules.config

In [7]:
from huggingface_hub import login
login(token=config.hugging_face_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/kellyshreeve/.cache/huggingface/token
Login successful


In [ ]:
start = time.time()

try:
  df_QA = pd.read_csv('/Users/kellyshreeve/Desktop/Data-Sets/Externship/qa_merged_clean.csv',
                    parse_dates=True)
except:
  from google.colab import drive
  drive.mount('/content/drive')
  df_QA = pd.read_csv('/content/drive/MyDrive/Externship/qa_merged_clean.csv')

end = time.time()

print(f'Computation Time: {end - start:.2f}')

Mounted at /content/drive
Computation Time: 18.50


In [ ]:
df_QA = pd.read_csv('/content/drive/MyDrive/Externship/qa_merged_clean.csv')

In [ ]:
df_QA.info()

In [ ]:
print(df_QA.isna().sum())

In [ ]:
df_QA=df_QA.reset_index(drop=True)

## **QA Models**

### Create Context

In [ ]:
context_short = df_QA.loc[0:100000, 'body_with_sentences_a'].dropna()
context_short = " ".join(context_short)

In [ ]:
print(f'Context Length: {len(context_short)} characters')

Context Length: 583419461 characters


In [ ]:
with open('/content/drive/MyDrive/Externship/context_short.txt', 'w') as file:
   file.write(context_shortest)

### Llama 2 Generative Chatbot from Context with Memory

In [ ]:
start = time.time()

model_id ='TheBloke/Llama-2-7B-chat-GGML'

llm = CTransformers(
  model=model_id,
  model_type='llama',
  max_new_tokens=512,
  temperature=0.5,
  repitition_penalty=1.1,
  top_k = 5,
  config={'context_length':700}
  )

with open('/content/drive/MyDrive/Externship/context_short.txt') as f:
    text = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)
texts = text_splitter.create_documents([text])

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                   model_kwargs={'device':'cuda'})

vectorstore = FAISS.from_documents(texts, embeddings)

custom_prompt_template='''Use the following pieces of information to answer the users question.
If you don't know the answer, please just say you don't know the answer. Don't make up an answer.
You are a helpful assistant, you always only answer for the assistant, then you stop.

Context:{context}
History:{history}
question:{question}

Only returns the helpful answer below and nothing else.
Helpful answer
'''

prompt = PromptTemplate(template=custom_prompt_template, input_variables=['history', 'context', 'question'])

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = retriever,
    chain_type_kwargs={'prompt':prompt,
                       "memory": ConversationBufferMemory(
                                memory_key="history",
                                input_key="question")}
)

end = time.time()

print(f'Computation Time: {end - start:.2f}')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Computation Time: 31.17


In [ ]:
# 7B-chat Model, top_k = 5, context_shortest.txt
start = time.time()

response = qa_chain.run('what is python?')

end = time.time()

print(response)
print()
print(f'Computation Time: {end - start:.2f}')

Python is a high-level, interpreted programming language with dynamic semantics, created by Guido van Rijn in 1991 at the University of Copenhagen.  It is known for its ease of use, flexibility, and a large and active development community. Python is often used as an all-purpose language, easily scaling up to meet the demands of larger projects, while still being easy to use for simple scripting tasks.

Computation Time: 390.86
